<a href="https://colab.research.google.com/github/raver8/nanolayer/blob/main/MANTEL_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Mantel Material Simulation (Zero-Cost Local vs. NIM API)
# ==============================================================================
# 1. ENVIRONMENT SETUP
# ==============================================================================
import subprocess
import sys


def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install MACE-torch (Zero-Cost Local Model) and ASE
print("Installing MACE and ASE... (This takes ~1 minute)")
install("mace-torch")
install("ase")

import torch
import numpy as np
import requests
from ase import Atoms
from ase.build import bulk, fcc111, add_adsorbate, hcp0001 # Added hcp0001
from ase.constraints import FixAtoms
from ase.optimize import BFGS
from mace.calculators import mace_mp
from google.colab import userdata
# The following line was removed as it caused a SecretNotFoundError if 'secretName' was not defined,
# and it's not essential for the default local model execution path.
# userdata.get('secretName')

# ==============================================================================
# 2. DEFINE THE MANTEL MATERIAL (B-Cu-Fe-Hf-Ti)
# ==============================================================================
def build_mantel_interface():
    """
    Constructs a representative interface of the Mantel Material.
    Layer 1: Hf-Fe 'Quasi-Steel' Matrix (Base)
    Layer 2: Titanium (Protective Coating)
    Dopants: Boron (Neutron Absorber) and Copper (Z-shielding)
    """
    print("\n🏗️ Building Mantel Material Heterostructure...")

    # 1. Base: Hafnium (HCP) - High-Z Shielding Layer
    # Using Hf lattice constants
    base_layer = bulk('Hf', 'hcp', a=3.19, c=5.05)
    base_layer = base_layer * (3, 3, 2)  # 3x3x2 Supercell

    # 2. Doping: Substitute random Hf atoms with Fe (Iron) to create the 'Quasi-Steel'
    # and Cu (Copper) for density.
    symbols = base_layer.get_chemical_symbols()
    num_sites = len(symbols)
    # Replace ~10% with Fe, ~5% with Cu
    import random
    random.seed(42) # For reproducibility
    indices = list(range(num_sites))
    random.shuffle(indices)

    for i in indices[:int(num_sites * 0.10)]:
        symbols[i] = 'Fe'
    for i in indices[int(num_sites * 0.10):int(num_sites * 0.15)]:
        symbols[i] = 'Cu'

    base_layer.set_chemical_symbols(symbols)

    # 3. Coating: Titanium (HCP) - Corrosion/Oxidation Shield
    # We construct a Ti slab and stack it on top
    # Replaced fcc111 with hcp0001 for correct HCP Titanium structure
    coating_layer = hcp0001('Ti', size=(3, 3, 2), a=2.95, c=4.68, vacuum=10.0)

    # 4. Interface: Stack Ti on top of Hf-Fe-Cu
    # Note: This creates a simplified interface. Real PVD/CVD is messier.
    # We adjust z-height to sit on top
    z_height = base_layer.positions[:, 2].max() + 2.5
    coating_layer.positions[:, 2] += z_height

    # Combine
    mantel_structure = base_layer + coating_layer

    # 5. Boron Doping (Interstitial)
    # Boron absorbs neutrons; usually sits in interstitial sites
    add_adsorbate(mantel_structure, 'B', height=1.2, position=(4, 4))
    add_adsorbate(mantel_structure, 'B', height=1.2, position=(2, 6))

    print(f"   Structure Created: {mantel_structure.get_chemical_formula()}")
    print(f"   Total Atoms: {len(mantel_structure)}")
    return mantel_structure

# ==============================================================================
# 3. SELECT CALCULATOR (LOCAL vs. NIM API)
# ==============================================================================

# --- OPTION A: ZERO-COST LOCAL (Running on Colab T4 GPU) ---
# This downloads the pre-trained MACE-MP-0 model (~150MB) and runs locally.
# Free. No API Key required.
def get_local_calculator():
    print("\n🚀 Initializing Local MACE-MP-0 Model (Zero Cost)...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"   Running on: {device.upper()}")

    # 'medium' maps to the standard MACE-MP-0 foundation model
    calc = mace_mp(model="medium", device=device, default_dtype="float32")
    return calc

# --- OPTION B: NVIDIA NIM API (Cloud Offload) ---
# Use this if your system is too large for the T4 or you want Reference Quality.
# Requires API Key from build.nvidia.com
class NvidiaNIMCalculator:
    def __init__(self, api_key, model_url):
        self.api_key = api_key
        self.url = model_url
        self.results = {}

    def get_potential_energy(self, atoms):
        # Construct Payload
        payload = {
            "structure": {
                "species": atoms.get_atomic_numbers().tolist(),
                "lattice": atoms.get_cell().tolist(),
                "coords": atoms.get_positions().tolist()
            },
            "options": {"properties": ["energy", "forces"]}
        }
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        # In a real script, handle retries/errors
        response = requests.post(self.url, json=payload, headers=headers)
        if response.status_code != 200:
            raise Exception(f"API Error: {response.text}")

        data = response.json()
        # Parse based on specific NIM output schema (simplified here)
        return data['energy']

# ==============================================================================
# 4. EXECUTION
# ==============================================================================

# 1. Build
atoms = build_mantel_interface()

# 2. Choose Mode (Defaulting to LOCAL for Free Tier)
USE_API = False # Set to True to use NVIDIA NIM
API_KEY = "nvapi-..." # REPLACE THIS
NIM_URL = "https://ai.api.nvidia.com/v1/materials/mace-mp-0" # CHECK DASHBOARD

if USE_API:
    atoms.calc = NvidiaNIMCalculator(API_KEY, NIM_URL)
else:
    atoms.calc = get_local_calculator()

# 3. Relax the Interface (Find stable geometry)
print("\n🧪 Starting Geometry Relaxation (Simulation)...")
print("   Objective: Minimize stress at Ti / Hf-Fe interface.")

# Fix the bottom layer to simulate bulk substrate
mask = [atom.index < 20 for atom in atoms]
constraint = FixAtoms(mask=mask)
atoms.set_constraint(constraint)

# Run BFGS Optimization
dyn = BFGS(atoms, trajectory='mantel_relaxation.traj')
dyn.run(fmax=0.05, steps=50) # Run 50 steps

print("\n✅ Simulation Complete!")
print(f"   Final Interface Energy: {atoms.get_potential_energy():.3f} eV")
print("   (Lower energy = More stable interface)")

# ==============================================================================
# 5. VISUALIZATION (Optional)
# ==============================================================================
# Use ASE GUI if running locally, or download .traj file from Colab files
# to view in OVITO.

Installing MACE and ASE... (This takes ~1 minute)

🏗️ Building Mantel Material Heterostructure...
   Structure Created: B2Cu2Fe3Hf31Ti18
   Total Atoms: 56

🚀 Initializing Local MACE-MP-0 Model (Zero Cost)...
   Running on: CPU
Cached MACE model to /root/.cache/mace/20231203mace128L1_epoch199model
Using Materials Project MACE for MACECalculator with /root/.cache/mace/20231203mace128L1_epoch199model
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.


/usr/local/lib/python3.12/dist-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['Default']
Default dtype float32 does not match model dtype float64, converting models to float32.

🧪 Starting Geometry Relaxation (Simulation)...
   Objective: Minimize stress at Ti / Hf-Fe interface.
      Step     Time          Energy          fmax
BFGS:    0 05:05:08     -328.487335      580.534302
BFGS:    1 05:05:11       95.778564     6648.959961
BFGS:    2 05:05:12      -38.117065     5904.338379
BFGS:    3 05:05:14      -62.565430     3897.955566
BFGS:    4 05:05:15     -203.764557     2747.554688
BFGS:    5 05:05:17     -368.399780      783.140869
BFGS:    6 05:05:18     -403.854858      515.262024
BFGS:    7 05:05:19     -642.604980     2254.362793
BFGS:    8 05:05:21     -807.889648     1260.959839
BFGS:    9 05:05:22     -898.845520     1466.313721
BFGS:   10 05:05:24     -970.392761     4233.875000
BFGS:   11 05:05:26    -1383.327393     4001.031738
BFGS:   12 05:05:27     -245.274078    10600.703125
BFGS:   13 05:05:29     -793.914673     9407.

In [ ]:
from ase.io import read, write

# Read the trajectory file we just created
traj_atoms = read('mantel_relaxation.traj', index=':')

# Save as Extended XYZ (Universally readable by OVITO)
write('mantel_relaxation.extxyz', traj_atoms)

print("✅ Conversion Complete! Download 'mantel_relaxation.extxyz' from the files tab.")

✅ Conversion Complete! Download 'mantel_relaxation.extxyz' from the files tab.
